In [1]:
import pygame
from pygame.locals import *
import sys
import math
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
ground = 0, 140, 100
road = 40, 40, 40
car_col = 150, 150, 150

In [3]:
cars = []

In [4]:
class Car:
    def __init__(self, speed=100, accel=1, target=100):
        self.x = -50
        self.y = 85
        self.w = 50
        self.max_speed = speed
        self.speed = self.max_speed
        self.accel = accel
        self.target = target
    
    def proc(self):
        if self.x < 0:
            self.w = 50 + self.x
        else:
            self.w = 50
        return (self.x, self.y, self.w, 30)
    
    def update(self, cars, fps):
        self.update_tail_speed(cars)
        if fps == 0:
            return
        self.x += self.speed / fps
    
    def update_tail_speed(self, cars):
        """
        If you're tailing a car, copy their speed.
        """
        lowest_tail_dist = math.inf
        lowest_tail_car = None
        for car in cars:
            if car.x > self.x:
                diff = car.x - (self.x + self.w)
                if diff < lowest_tail_dist:
                    lowest_tail_dist = diff
                    lowest_tail_car = car
        if lowest_tail_dist < 10:
            self.speed = lowest_tail_car.speed
        else:
            if self.max_speed < self.target:
                self.max_speed += self.accel
            self.speed = self.max_speed

In [5]:
def spawn_car(speed=100, accel=1, target=100):
    cars.append(Car(speed, accel, target))

In [6]:
def update_all_cars(fps, screen_edge):
    for car in cars:
        car.update(cars, fps)
    new_cars = []
    for car in cars:
        if car.x < screen_edge:
            new_cars.append(car)
    return new_cars

In [7]:
def mainloop(screen):
    global cars
    clock = pygame.time.Clock()
    click_last = False
    prev_tick = time.time()
    next_speed = 100
    spawn_time = 1
    while 1:
        time.sleep(0.01)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.display.quit()
                pygame.quit()
                return
        clock.tick()
        screen.fill(ground)
        
        # Road
        screen.fill(road, (0, 80, 750, 40))
        
        # Cars
        for car in cars:
            screen.fill(car_col, car.proc())
        
        cars = update_all_cars(clock.get_fps(), 750)
        
        tick_time = time.time() - prev_tick
        if tick_time >= spawn_time:
            prev_tick += tick_time
            spawn_car(next_speed, 0.1, next_speed)
            next_speed = 175
        
        left = pygame.mouse.get_pressed(3)[0]
        if left != click_last:
            click_last = left
            if left == True:
                next_speed = 100
        pygame.display.flip()

In [8]:
def initialize_screen(screen_size):
    return pygame.display.set_mode(screen_size)

In [9]:
def main():
    screen_size = width, height = 750, 200
    scrn = initialize_screen(screen_size)
    mainloop(scrn)

In [10]:
main()